In [108]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.model_selection import cross_val_score

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Подход 1 - Градиентный бустинг

In [96]:
features = pd.read_csv("/Users/semyon/python_tasks/final_task/features.csv", index_col="match_id")
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [97]:
future = ["duration", "radiant_win", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", 
          "barracks_status_dire"]

X = features.drop(future, axis=1)
y = features["radiant_win"]

In [99]:
headers = X.columns.values

for i in range(len(headers)):
    X[headers[i]].fillna(0, inplace=True)

X = X.values
y = y.values

In [114]:
import datetime
import time

kf = KFold(n=len(X), n_folds=5, random_state=1, shuffle=True)
n_est = [10, 20, 30, 40, 50, 60]

for i in range(len(n_est)):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est[i], random_state=1)
    results = cross_val_score(X=X, y=y, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(n_est[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))


10: 0.6648329228049101
Time elapsed: 0:00:35.397402

20: 0.6821140369500347
Time elapsed: 0:01:06.620759

30: 0.6896947542059905
Time elapsed: 0:01:36.837552

40: 0.6939341916023564
Time elapsed: 0:02:08.234120

50: 0.6971139526079494
Time elapsed: 0:02:55.616751

60: 0.6998001572455899
Time elapsed: 0:03:44.431056



### Отберем случайным образом n объектов каждого признака, где m - доля отобранных объектов от всей выборки

In [135]:
import random 

In [176]:
future_new = ["duration", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]

X_y = features.drop(future_new, axis=1)
headers = X_y.columns.values

In [177]:
headers == "radiant_win"

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True], dtype=bool)

In [178]:
for i in range(len(headers)):
    X_y[headers[i]].fillna(0, inplace=True)

In [180]:
X_y = X_y.values

In [183]:
X_y[:, -1]

array([ 1.,  1.,  0., ...,  0.,  0.,  1.])

In [184]:
m = 0.5
n = m*len(X)

random.shuffle(X_y)
X_y_new = X_y[:int(n+1)]

In [212]:
y_new = X_y[:, -1]
X_new = X_y[:, :-1]

In [214]:
kf = KFold(n=len(X_new), n_folds=5, random_state=1, shuffle=True)
n_est = [10, 20, 30, 40, 50, 60]

for i in range(len(n_est)):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est[i], random_state=1)
    results = cross_val_score(X=X_new, y=y_new, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(n_est[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

10: 0.6851643197936085
Time elapsed: 0:00:35.670981

20: 0.7052813448511153
Time elapsed: 0:01:14.464681

30: 0.713863292880709
Time elapsed: 0:01:43.861978

40: 0.7207310836769188
Time elapsed: 0:02:12.297554

50: 0.7255527547320174
Time elapsed: 0:02:51.386903

60: 0.7297538259540607
Time elapsed: 0:02:55.461449



### Уменьшим max_depth с 3 до 2

In [216]:
kf = KFold(n=len(X_new), n_folds=5, random_state=1, shuffle=True)
n_est = [10, 20, 30, 40, 50, 60]

for i in range(len(n_est)):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est[i], random_state=1, max_depth=2)
    results = cross_val_score(X=X_new, y=y_new, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(n_est[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

10: 0.6656286838235581
Time elapsed: 0:00:22.067859

20: 0.6917615339129851
Time elapsed: 0:00:42.981699

30: 0.7010148673857927
Time elapsed: 0:01:01.781547

40: 0.7068149050885528
Time elapsed: 0:01:24.305677

50: 0.710778484561019
Time elapsed: 0:01:42.759615

60: 0.7144334065487709
Time elapsed: 0:02:09.528212



### Уменьшим max_depth с 2 до 1

In [288]:
kf = KFold(n=len(X_new), n_folds=5, random_state=1, shuffle=True)
n_est = [10, 20, 30, 40, 50, 60]

for i in range(len(n_est)):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est[i], random_state=1, max_depth=1)
    results = cross_val_score(X=X_new, y=y_new, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(n_est[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

10: 0.6250293337473546
Time elapsed: 0:00:13.889928

20: 0.6603082389522188
Time elapsed: 0:00:23.182723

30: 0.678855373539748
Time elapsed: 0:00:33.541344

40: 0.6870932058498579
Time elapsed: 0:00:43.170735

50: 0.6932133219433472
Time elapsed: 0:00:52.798604

60: 0.697132872793059
Time elapsed: 0:01:02.639879



### Найдем все столбцы с пропусками в исходных данных
### (Будем считать не количество пустых ячеек, а их долю)

In [285]:
all_headers = features.columns.values
empty_cells_dict = dict()

def empty_cells(column):
    notempty = features[column].count()
    overall = len(features[column])
    empty_cells_dict[column] = float(round((overall - notempty)/overall, 3))

In [286]:
for i in range(len(all_headers)):
    empty_cells(all_headers[i])

In [287]:
import operator

sorted_dict = sorted(empty_cells_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_dict[:12]

[('first_blood_player2', 0.452),
 ('radiant_flying_courier_time', 0.283),
 ('dire_flying_courier_time', 0.268),
 ('first_blood_time', 0.201),
 ('first_blood_team', 0.201),
 ('first_blood_player1', 0.201),
 ('dire_bottle_time', 0.166),
 ('radiant_bottle_time', 0.161),
 ('radiant_first_ward_time', 0.019),
 ('dire_first_ward_time', 0.019),
 ('radiant_courier_time', 0.007),
 ('dire_courier_time', 0.007)]

# Отчет

### 1. Следуюшие признаки имеют пропуски в своих значениях:
 
### ('first_blood_player2', 0.452),
### ('radiant_flying_courier_time', 0.283),
### ('dire_flying_courier_time', 0.268),
### ('first_blood_time', 0.201),
### ('first_blood_team', 0.201),
### ('first_blood_player1', 0.201),
### ('dire_bottle_time', 0.166),
### ('radiant_bottle_time', 0.161),
### ('radiant_first_ward_time', 0.019),
### ('dire_first_ward_time', 0.019),
### ('radiant_courier_time', 0.007),
### ('dire_courier_time', 0.007)
 
### Например, существенная доля пропусков в признаке "first_blood_player1" может быть связана с тем, что в первые 5 минут матча не произошло события first blood. Или же пропуски в признаке "radiant_bottle_time" могут быть объяснены тем, что команда вплоть до исхода матча не приобрела предмет "bottle".
 
### 2. Целевая переменная: "radiant_win"

### 3. Кросс-валидация для градиентного бустинга с 30 деревьями и дефолтными параметрами:

### a) длительность: 01 мин 36.84 сек

### б) качество AUC_ROC: 0.6896947542059905

### 4. Использование больше 30 деревьев:

### Оправдано при уменьшении max_depth, например с 3 до 1, при этом процесс ускоряется в 3 раза, а качество (при одинаковых количествах деревьев) уменьшается незначительно.

# Подход 2 - Логистическая регрессия

In [448]:
future = ["duration", "radiant_win", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]

X = features.drop(future, axis=1)
y = features["radiant_win"]

headers = X.columns.values

for i in range(len(headers)):
    X[headers[i]].fillna(0, inplace=True)

X = X.values
y = y.values

In [449]:
from sklearn.linear_model import LogisticRegression 

In [450]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [451]:
C_array = np.power(10.0, np.arange(-6, 1))

for i in range(len(C_array)):
    start_time = datetime.datetime.now()
    kf = KFold(n=len(X_scaled), n_folds=5, random_state=1, shuffle=True)
    clf = LogisticRegression(C=C_array[i], random_state=1, penalty="l2", solver="sag")
    results = cross_val_score(X=X_scaled, y=y, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(C_array[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

1e-06: 0.6882248200389643
Time elapsed: 0:00:07.410762

1e-05: 0.6951715154395941
Time elapsed: 0:00:07.295226

0.0001: 0.7112812236746439
Time elapsed: 0:00:07.371688

0.001: 0.7162097438628422
Time elapsed: 0:00:08.233321

0.01: 0.7163757705375227
Time elapsed: 0:00:11.113963

0.1: 0.7163507985577163
Time elapsed: 0:00:11.196695

1.0: 0.7163472913397035
Time elapsed: 0:00:10.478385



### Уберем категориальные признаки из выборки

In [452]:
headers = features.columns.values
headers

array(['start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp',
       'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero',
       'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths',
       'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh',
       'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp',
       'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero',
       'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths',
       'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh',
       'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp',
       'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero',
       'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths',
       'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh',
       'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp',
       'd5_gold', 'd5_lh', 'd5_kills', 'd5_death

In [454]:
categorical = ["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero", "d1_hero", "d2_hero", "d3_hero", 
               "d4_hero", "d5_hero"]

X = features.drop(future, axis=1)
X_noncategorical = X.drop(categorical, axis=1)
y = features["radiant_win"]

for i in range(len(X_noncategorical.columns.values)):
    X_noncategorical[X_noncategorical.columns.values[i]].fillna(0, inplace=True)

X_noncategorical = X_noncategorical.values
y = y.values

scaler = StandardScaler()
scaler.fit(X_noncategorical)
X_noncategorical_scaled = scaler.transform(X_noncategorical)

In [456]:
C_array = np.power(10.0, np.arange(-6, 1))

for i in range(len(C_array)):
    start_time = datetime.datetime.now()
    kf = KFold(n=len(X_noncategorical_scaled), n_folds=5, random_state=1, shuffle=True)
    clf = LogisticRegression(C=C_array[i], random_state=1, penalty="l2", solver="sag")
    results = cross_val_score(X=X_noncategorical_scaled, y=y, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(C_array[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

1e-06: 0.6881706233170358
Time elapsed: 0:00:06.637971

1e-05: 0.6951059856286517
Time elapsed: 0:00:07.195328

0.0001: 0.711259204203986
Time elapsed: 0:00:07.698114

0.001: 0.7162335431953399
Time elapsed: 0:00:07.545725

0.01: 0.716408681728097
Time elapsed: 0:00:09.358952

0.1: 0.7163839265439949
Time elapsed: 0:00:09.644508

1.0: 0.7163802499786964
Time elapsed: 0:00:09.617594



### Сколько разных героев существует

In [458]:
N = len(np.unique(features[categorical]['d1_hero']))
data = features[categorical].drop(["lobby_type"], axis=1)

In [459]:
headers = data.columns.values
data[:5]

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero
match_id,,,,,,,,,,
0,11,67,29,20,105,4,42,21,37,84
1,42,49,67,37,26,39,88,79,7,12
2,33,98,20,27,4,22,66,86,29,80
3,29,30,75,37,41,96,48,15,102,20
4,13,27,30,72,93,26,69,22,25,8


### Соберем мешок слов из уникальных героев

In [460]:
hero_D = np.unique(data['d1_hero'])

N = max(hero_D)

X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1 

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [461]:
unity = 0
minus = 0
zeros = 0

for i in range(len(X_pick)):
    for k in range(len(X_pick[i])):
        if X_pick[i][k] == 1:
            unity += 1
        elif X_pick[i][k] == -1:
            minus += 1
        else:
            zeros += 1
            
print("1: {}\n-1: {}\n0: {}".format(unity, minus, zeros))

1: 486150
-1: 486150
0: 9917460


In [462]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
selector.fit_transform(X_pick)

X_new = np.hstack((X_noncategorical_scaled, X_pick))

In [465]:
C_array = np.power(10.0, np.arange(-6, -1))

for i in range(len(C_array)):
    start_time = datetime.datetime.now()
    kf = KFold(n=len(X_new), n_folds=5, random_state=1, shuffle=True)
    clf = LogisticRegression(C=C_array[i], random_state=1, penalty="l2", solver="sag")
    results = cross_val_score(X=X_new, y=y, estimator=clf, cv=kf, scoring="roc_auc")
    print("{}: {}".format(C_array[i], results.mean()))
    print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

1e-06: 0.6905621378311867
Time elapsed: 0:00:11.601466

1e-05: 0.6991832436844884
Time elapsed: 0:00:11.469015

0.0001: 0.7249773458984026
Time elapsed: 0:00:11.706079

0.001: 0.7462115671043215
Time elapsed: 0:00:13.786981

0.01: 0.7516522504802192
Time elapsed: 0:00:20.881597



## Проверим лучшую модель на тестовой выборке

In [569]:
X_test = pd.read_csv("/Users/semyon/python_tasks/final_task/features_test.csv", index_col="match_id")
X_test[:5]

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [570]:
hero_D = np.unique(X_test['d1_hero'])

N = max(hero_D)

X_pick = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [571]:
categorical = ["lobby_type", "r1_hero", "r2_hero", "r3_hero", "r4_hero", "r5_hero", "d1_hero", "d2_hero", "d3_hero", "d4_hero", "d5_hero"]

X_test_noncategorical = X_test.drop(categorical, axis=1)

for i in range(len(X_test_noncategorical.columns.values)):
    X_test_noncategorical[X_test_noncategorical.columns.values[i]].fillna(0, inplace=True)

X_test_noncategorical = X_test_noncategorical.values
scaler = StandardScaler()
scaler.fit(X_test_noncategorical)
X_test_noncategorical_scaled = scaler.transform(X_test_noncategorical)

X_test_scaled = np.hstack((X_test_noncategorical_scaled, X_pick))

In [579]:
X_train = X_new
start_time = datetime.datetime.now()
clf_logit = LogisticRegression(C=.01, random_state=1, penalty="l2", solver="sag")
clf_logit.fit(X_train, y)
predictions = clf_logit.predict_proba(X_test_scaled)[:, 1]
print('Time elapsed: {}\n'.format(datetime.datetime.now() - start_time))

print("\tradiant_win\nmin: {}\nmax: {}".format(min(predictions), max(predictions)))

Time elapsed: 0:00:05.351809

	radiant_win
min: 0.008447702602170981
max: 0.9959849793376752


# Отчет

## 1. Лучшее качество на исходных признаках: 0.7163757705375227 при С=0.01; Оно в среднем выше качества ГБ (~0.75 против ~0.71) и занимает на  порядок меньше времени (единицы секунд против единиц минут); это связано с тем, что логистическая регрессия является представлителем  линейных методов, которые, как правило, работают гораздо быстрее композиций деревьев. 
## 2. Удаление категориальных признаков дает улучшение качества в четвертом знаке после запятой:
## для исходных данных при C=0.01: 0.7163757705375227;
## без категориальных признаков при C=0.01: 0.716408681728097;
## Это связано с тем, что исходные категориальные признаки не несут в себе однозначной зависимости, поэтому воспринимаются классификатором как шумовые данные. 
## 3. В игре существуют 112 различных идентификаторов героев, однако представлены в выборке только 108. 
## 4. Герои являются важными признаками в игре, поэтому соответствующие им категориальные признаки были преобразованы в более информативные признаки с помощью метода "мешка слов". С учетом новых признаков качество значительно улучшилось с 0.716408681728097 до 0.7516522504802192
## 5. Минимальное и максимальное значение прогноза лучшего алгоритма:
## min: 0.008447702602170981
## max: 0.9959849793376752